In [1]:
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2

In [2]:
data = np.loadtxt('grade.csv',delimiter=',')
row = data.shape[0]
col = data.shape[1]

label = data[:,col-3]
data = data[:,:col-3]

one = 0
two = 0
three = 0
four = 0
five = 0

for i in range(row):
    if label[i] <= 4:
        label[i] = 1
        one += 1
    elif label[i] <= 8:
        label[i] = 2
        two += 1
    elif label[i] <= 12:
        label[i] = 3
        three += 1
    elif label[i] <= 16:
        label[i] = 4
        four += 1
    elif label[i] <= 20:
        label[i] = 5
        five += 1
print(one,two,three,four,five, row)

2 109 156 109 19 395


In [3]:
train, test, train_label_1, test_label_1 = train_test_split(data, label, test_size=0.25, random_state = 58)
train_row = train.shape[0]
test_row = test.shape[0]

In [4]:
select_top_k = SelectKBest(score_func=chi2, k = 7)
fit = select_top_k.fit(train, train_label_1)

In [5]:
sel_train = fit.transform(train)
sel_test = fit.transform(test)

idx = 1
idxs = []
for i in fit.get_support():
    if i == True:
        idxs.append(idx)
    idx += 1
print(idxs)

sel_train = preprocessing.scale(sel_train)
sel_test = preprocessing.scale(sel_test)

[6, 9, 12, 14, 15, 16, 28]


In [6]:
sel_clf = svm.SVC(random_state = 58)
sel_clf.fit(sel_train, train_label_1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=58, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
c = 0
for i in range(train_row):
    if sel_clf.predict([sel_train[i,:]]) != train_label_1[i]:
        #print(i,clf.predict([train[i,:]]),train_label_1[i])
        c += 1
print(1-c/row)

c = 0
for i in range(test_row):
    if sel_clf.predict([sel_test[i,:]]) != test_label_1[i]:
        #print(i,clf.predict([test[i,:]]),test_label_1[i])
        c += 1
print(1-c/test_row)

0.6531645569620252
0.43434343434343436


In [8]:
param_grid = {'C':[0.1, 1, 10, 100], 'gamma':[1, 0.1, 0.01, 0.001]}
grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose = 2)

In [9]:
grid.fit(sel_train, train_label_1)
grid_predictions = grid.predict(sel_test)
print(classification_report(test_label_1,grid_predictions))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ...........

C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ................................. C=10, gamma=0.01, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ................................. C=10, gamma=0.01, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] ................................. C=10, gamma=0.01, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................................ C=10, gamma=0.001, total=   0.0s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................................ C=10, gamma=0.001, total=   0.0s
[CV] C

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.3s finished
C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
